In [1]:
%load_ext autoreload
%autoreload 1

import sys

sys.path.append("../utils/")

import os
import json
import pdb
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

from work_crawler import save_to_mlab

%aimport work_crawler

In [9]:
for root, dirs, files in os.walk("../json"):
    composers = [file.split(".")[0] for file in files]
    

In [10]:
composers

['cpebach',
 'puccini',
 'schutz',
 'gluck',
 'ravel',
 'mendelssohn',
 'gabrieli',
 'rameau',
 'ives',
 'wagner',
 'gibbons',
 'smetana',
 'donizetti',
 'chopin',
 'ockeghem',
 'morley',
 'palestrina',
 'gounod',
 'berlioz',
 'mussorgsky',
 'prokofiev',
 'rimskykorsakov',
 'couperin',
 'borodin',
 'dscarlatti',
 'byrd',
 'stamitz',
 'sibelius',
 'jsbach',
 'lully',
 'wolf',
 'dufay',
 'faure',
 'schumann',
 'vivaldi',
 'saintsaens',
 'offenbach',
 'elgar',
 'varese',
 'franck',
 'schubert',
 'ascarlatti',
 'mozart',
 'grieg',
 'weelkes',
 'rstrauss',
 'dowland',
 'tallis',
 'caccini',
 'telemann',
 'praetorius',
 'haydn',
 'bartok',
 'meyerbeer',
 'brahms',
 'frescobaldi',
 'machaut',
 'lassus',
 'rossini',
 'mahler',
 'tchaikovsky',
 'debussy',
 'rachmaninoff',
 'hindemith',
 'bizet',
 'binchois',
 'handel',
 'monteverdi',
 'bruckner',
 'josquin',
 'purcell',
 'devitry',
 'victoria',
 'berg',
 'satie',
 'pergolesi',
 'schoenberg',
 'liszt',
 'beethoven',
 'scriabin',
 'vaughanwilliam

In [3]:
def is_bytes(resp):
    try:
        resp.content.decode()
        return True
    except:
        return False

In [4]:
class ProcessComposer():
    
    def __init__(self, composer):
        composer_json = self._get_json(composer)
        self._create_composer_df(composer_json)
    
    def _get_json(self, composer):
        with open(f"../json/{composer}.json", "r") as file:
            composer_json = json.load(file)
        return composer_json
    
    def _create_composer_df(self, composer_json):
        composer_df = pd.DataFrame(composer_json)
        composer_df['score_ids'] = composer_df.scores.apply(lambda score_lst: [int(x.split("/")[-1]) for x in score_lst])
        self.composer_df = composer_df
       
    def _process_work(self, work):
        pdfs = []
        for score_id in work.score_ids:
            resp = requests.get(f"https://imslp.org/wiki/Special:IMSLPDisclaimerAccept/{score_id}", headers={"X-Forwarded-For": "24.18.100.31"})
            try:
#                 if is_bytes(resp):
#                     print("bytes")
#                     pdfs.append(resp.url)
#                 else:
#                 print("string")
                soup = BeautifulSoup(resp.content)
                pdfs.append(soup.find("span", {"id":"sm_dl_wait"}).get("data-id"))
            except:                
                time.sleep(2)
                continue
            
            time.sleep(2)
    
        return pdfs
    
    def _save_work(self, work_dict):
        save_to_mlab(work_dict)
    
    def run_work_retrieval(self, start=0, stop=-1):
        for i, row in self.composer_df[start:stop].iterrows():
            pdfs = self._process_work(row)
            row_dict = row.to_dict()
            
            row_dict["pdfs"] = pdfs
            print(row_dict)
            if len(pdfs) > 0:
                self._save_work(row_dict)
            

In [6]:
cp = ProcessComposer("jsbach")
# cp.composer_df


In [7]:
cp.run_work_retrieval(7, 8)

{'composer': 'Bach, Johann Sebastian', 'date': '1715', 'scores': ['https://imslp.org/wiki/Special:ImagefromIndex/01424'], 'tags': ['Sacred cantatas', 'Cantatas', 'Religious works', 'For 4 voices, mixed chorus, orchestra', 'For voices and chorus with orchestra'], 'title': 'Ach! ich sehe, itzt, da ich zur Hochzeit gehe, BWV 162', 'score_ids': [1424], 'pdfs': ['http://ks.imslp.net/files/imglnks/usimg/7/77/IMSLP01424-BWV0162.pdf']}


In [ ]:
work = cp.composer_df.iloc[847]
print(cp.process_work(work))

In [ ]:
def process_json(json_files):
    for file in json_files:
        with open(f"../json/{file}", "r") as composer:
            composer_data = json.load(composer)

        composer_df = pd.DataFrame(composer_data)
        composer_df['score_ids'] = composer_df.scores.apply(lambda score_lst: [int(x.split("/")[-1]) for x in score_lst])

        for ids in composer_df.score_ids:
            for score_id in ids:
                print(score_id)
                try:
                    content = requests.get(f"https://imslp.org/wiki/Special:IMSLPDisclaimerAccept/{score_id}", headers={"X-Forwarded-For": "24.18.100.31"}).content

                with open("../index.html", "w") as file:
                    file.write(content.decode("utf-8"))
                except:
                    time.sleep(2)
                    continue
                time.sleep(2)
        break
        
process_json(json_files)

In [ ]:
with open("../json/beethoven.json", "r") as file:
    data = json.load(file)

In [ ]:
composer_df = pd.DataFrame(data)
composer_df.iloc[0].scores

In [ ]:
composer_df['score_ids'] = composer_df.scores.apply(lambda score_lst: [int(x.split("/")[-1]) for x in score_lst])

composer_df.iloc[0].to_dict()

In [ ]:
content = requests.get("https://imslp.org/wiki/Special:IMSLPDisclaimerAccept/51737", headers={"X-Forwarded-For": "24.18.100.31"}).content

with open("../index.html", "w") as file:
    file.write(content.decode("utf-8"))

In [ ]:
requests.get("http://127.0.0.1:8080", headers={"X-Forwarded-For": "24.18.100.31"})

In [ ]:
!curl --header "X-Forwarded-For: 192.168.0.99" https://imslp.org/wiki/Special:ImagefromIndex/51737 > ../index.html